<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/FSML_Python/chap07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第7章 サポートベクトルマシン

## 7.1 マージンを最大とする識別面を求める

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import make_classification
from sklearn.svm import SVC

乱数を使って、データを生成
* 2クラス
* 特徴ベクトルは2次元（冗長な次元はなし）
* 各クラスにつき一つの正規分布からデータを生成
* ランダムにクラスを反転させることは行わない
* クラスが適度に分かれているように調整

In [ ]:
X, y = make_classification(n_features=2, n_redundant=0, n_clusters_per_class=1, flip_y=0, class_sep=2.0, random_state=2)
plt.plot(X[y==0,0], X[y==0,1],"bs")
plt.plot(X[y==1,0], X[y==1,1],"r^")
plt.show()

線形カーネル(linear)は、この特徴空間でマージン最大化を行うものです。

In [ ]:
clf = SVC(kernel='linear')
clf.fit(X, y)

識別面を表示します。各クラスのサポートベクトルと識別面の距離が等しくなっているのがわかります。

In [ ]:
plot_decision_regions(X=X, y=y, clf=clf, legend=2)
plt.show()

## 7.2 ソフトマージンによる誤識別データの吸収

クラスの分離度(class_sep)の値を少し小さくして、やや難しいデータを作成し、同様の手順でSVMによる識別を行います。線型分離不可能なデータなので、スラック変数の重み$C$を引数で与えます。

In [ ]:
X, y = make_classification(n_features=2, n_redundant=0, n_clusters_per_class=1, flip_y=0, class_sep=1.0, random_state=2)
plt.plot(X[y==0,0], X[y==0,1],"bs")
plt.plot(X[y==1,0], X[y==1,1],"r^")
plt.show()

In [ ]:
clf = SVC(kernel='linear', C=1)
clf.fit(X, y)

In [ ]:
plot_decision_regions(X=X, y=y, clf=clf, legend=2)
plt.show()

## 7.3 カーネル関数を用いたSVM

非線形識別面での識別を行うので、さらにデータを複雑なものにします。

In [ ]:
X, y = make_classification(n_features=2, n_redundant=0, n_clusters_per_class=1, class_sep=0.5, random_state=3)
plt.plot(X[y==0,0], X[y==0,1],"bs")
plt.plot(X[y==1,0], X[y==1,1],"r^")
plt.show()

多項式カーネル

In [ ]:
clf = SVC(kernel='poly', degree=3, C=1)
clf.fit(X, y)

In [ ]:
plot_decision_regions(X=X, y=y, clf=clf, legend=2)
plt.show()

RBFカーネル

In [ ]:
clf = SVC(kernel='rbf', gamma=1, C=1)
clf.fit(X, y)

In [ ]:
plot_decision_regions(X=X, y=y, clf=clf, legend=2)
plt.show()

## 7.4 ハイパーパラメータのグリッドサーチ

In [ ]:
from sklearn.model_selection import GridSearchCV

「スラック変数の重みC」と「多項式カーネルの次数degree」の組み合わせでGridSearchを行います。

パラメータとして多項式カーネルを与えて、サポートベクトルマシン[SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)のインスタンスsvcを作成します。

In [ ]:
svc = SVC(kernel='poly')
svc

GridSearchを行います。「スラック変数の重みC」と「多項式カーネルの次数degree」の組み合わせで、リストを値とするディクショナリの配列param\_gridを作成します。

In [ ]:
param_grid = [
  {'C': [0.1, 1, 10, 100, 1000], 'degree': [1,2,3]}
 ]

識別器のインスタンスを第1引数、グリッドを第2引数として、[GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)のインスタンスを作成し、fitメソッドを実行します。パラメータcvの値がNone（デフォルト）のとき、性能は5-fold CVで評価されます。許容できる実行時間を考えて、この値を調整します。

In [ ]:
clf = GridSearchCV(svc, param_grid, cv=3)
clf.fit(X, y)

結果はcv\_results\_属性の値として、辞書型で得られます。また、容易にpandasのDataFrame型に変換できます。

In [ ]:
df = pd.DataFrame(clf.cv_results_)
df

すべてのパラメータの組み合わせについて、スコアを表示します。

In [ ]:
re = clf.cv_results_
for params, mean_score, std_score in zip(re['params'], re['mean_test_score'], re['std_test_score']):
    print(f"{mean_score:.3f} (+/- {std_score*2:.3f}) for {params}")

性能が最大となるパラメータとそのときのスコアを表示します。

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

ガウス過程回帰の説明

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import gaussian_process
from sklearn.gaussian_process import kernels

In [ ]:
X = np.array([-4, -3, -1, 0, 2])
y = np.array([-2, 0,-1, 2, -1])
xs = np.linspace(-5, 5, 101)
kernel = kernels.RBF(length_scale=np.sqrt(2)) + kernels.WhiteKernel(noise_level=0.05)

In [ ]:
gp = gaussian_process.GaussianProcessRegressor(kernel=kernel, optimizer=None)
gp.fit(X[:, None], y)
mu, std = gp.predict(xs[:, None], return_std=True)

In [ ]:
plt.figure(figsize=(4,3))
plt.plot(xs, mu)
for i in [3, 2, 1]:
  plt.fill_between(xs, mu+i*std, mu-i*std, color=str(0.5+0.1*i))
plt.scatter(X, y)
plt.show()

Optuna

In [ ]:
!pip install optuna

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import optuna

In [ ]:
X, y = load_diabetes(return_X_y=True, as_frame=True)

In [ ]:
def objective(trial):
  gamma = trial.suggest_float('gamma', 1e-3, 1e3)
  C = trial.suggest_float('C', 1e-3, 1e3)
  reg = SVR(kernel='rbf', gamma=gamma, C=C)
  score = cross_val_score(reg, X, y, cv=3, scoring="r2")
  r2_mean = score.mean()
  return r2_mean

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=30)
print('params:', study.best_params)

In [ ]:
optuna.visualization.plot_contour(study)

## 課題

scikit-learn付属の wine データに対する識別をRBFカーネルを用いて行い、識別結果を図示せよ。その際に、C = 1, 100, 10000 として結果がどう変わるかを調べよ。


### 解答例


SVMは2値分類問題に適用可能な識別器なので、多値分類に用いるときは、以下のいずれかの方法をとる必要があります。

* one-versus-rest法 ('ovr')
  * 各クラスについて、そのクラスに属するかどうかを識別するSVMを作る
  * ２つ以上のクラスに属すると判定された場合は識別面からの距離が大きいものに分類する
* ペアワイズ法 ('ovo')
  * クラス対ごとに識別器を作る
  * 判定は多数決を取る

scikit-learnではSVCの引数decision_function_shapeで指定し、デフォルトは 'ovr' です

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
wine = load_wine()
X = wine.data
y = wine.target
print(X)
print(y)

本来 X は13次元ありますが、可視化のため２次元に圧縮します。(X --> X2)

In [ ]:
pca = PCA(n_components=2)
X2 = pca.fit_transform(X)

識別面のおおまかなイメージを確認します。

In [ ]:
plt.plot(X2[y==0,0], X2[y==0,1],"bs", label=wine.target_names[0])
plt.plot(X2[y==1,0], X2[y==1,1],"r^", label=wine.target_names[1])
plt.plot(X2[y==2,0], X2[y==2,1],"go", label=wine.target_names[2])
plt.legend()
plt.show()

In [ ]:
# ここからは自力で